## Segmenting and Clustering in Toronto

First lets get data

In [1]:
import requests
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 
import folium # map rendering library
!pip install BeautifulSoup4 mrjob pattern seaborn
import lxml
from bs4 import BeautifulSoup
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.42 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.85 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  24.88 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.62 MB/s
Requirement not upgraded as not directly required: BeautifulSoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
    100% |████████████████████████████████| 317kB 2.4MB/s eta 0:00:01
    100% |████████████████████████████████| 22.3MB 42kB/s  eta 0:00:01    83% |██████████████████████████▊     | 18.6MB 32.7MB/s 

In [2]:
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
!wget -q -O 'wiki.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [3]:
with open("wiki.html") as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [4]:
table=soup.find('table').text
table
post_d=table.split('\n')

We have two empty value between each row, let's delete first 2 empty values

In [5]:
del(post_d[0])
del(post_d[0])

In [6]:
#function devides lists to n parts
from math import ceil
def parting(xs, parts):
    part_len = ceil(len(xs)/parts)
    return [xs[part_len*k:part_len*(k+1)] for k in range(parts)]

In [7]:
len(post_d)/5 # in each row we have 3 values and 2 empty - 5

290.0

In [8]:
k=parting(post_d,290)

In [14]:
#Create Dataframe with all values we need
post_df=pd.DataFrame(k,columns=['Postal Code', 'Borough','Neighbourhood','A','B'])
del(post_df['A'])
del(post_df['B'])
post_df.drop([0],inplace=True)

post_df.replace("Not assigned",np.nan,inplace=True)
post_df.dropna(subset=["Borough"],axis=0,inplace=True)

In this dataframe there are duplicates and we should combine neighbourhood of the same Postal Code

In [10]:
p_l=post_df.groupby(['Postal Code','Borough']).apply(lambda x: x['Neighbourhood'].tolist())

In [11]:
df=pd.DataFrame(p_l)
df.reset_index(['Postal Code','Borough'],inplace=True)
df.columns
df.columns = ['Postal Code','Borough','Neighbourhood']

In [12]:
coo_url='http://cocl.us/Geospatial_data'
coo_df=pd.read_csv(coo_url)
coo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
data_df=pd.merge(df, coo_df, on='Postal Code', how='outer')
data_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848
